## 수치 미분을 이용한 심층 신경망 학습

In [1]:
import time
import numpy as np

## 유틸리티 함수

In [2]:
def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

## Sigmoid 구현

In [6]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1

    def __call__(self, x):
        self.last_o = 1 / (1.0 + np.exp(-x))
        return self.last_o

    def grad(self): # sigmoid(x)(1 - sigmoid(x))
        return self.last_o * (1 - self.last_o)

## Mean Squared Error 구현

In [7]:
class MeanSquaredError:
    def __init__(self):
        # To calculate gradient
        self.dh = 1
        self.last_diff = 1

    def __call__(self, h, y): # 1 / 2 * mean((h - y)^2)
        self.last_diff = h - y
        return 1 / 2 * np.mean(np.square(h - y))

    def grad(self): # h - y
        return self.last_diff

## 뉴런 구현

In [8]:
class Neuron:
    def __init__(self, W, b, a_obj):
        # Model Parameters
        self.W = W
        self.b = b
        self.a = a_obj()
        
        # gradient
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W))
        
        self.last_x = np.zeros((self.W.shape[0]))
        self.last_h = np.zeros((self.W.shape[1]))

    def __call__(self, x):
        self.last_x = x
        self.last_h = _m(_t(self.W), x) + self.b
        return self.a(self.last_h)

    def grad(self):
        return self.W * self.a.grad()

    def grad_W(self, dh):
        grad = np.ones_like(self.W)
        grad_a = self.a.grad()
        
        for j in range(grad.shape[1]):
            grad[:, j] = dh[j] * grad_a[j] * self.last_x
        return grad

    def grad_b(self, dh):
        return dh * self.a.grad()

## 심층신경망 구현

In [9]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, input, output, activation=Sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))

        # Hidden Layers
        for index in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output Layer
        W, b = init_var(num_neuron, output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, loss_obj):
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj)
        
        # Back-propagation
        for i in range(len(self.sequence)- 1, 0, -1):
            l1 = self.sequence[i]
            l0 = self.sequence[i - 1]
            
            l0.dh = _m(l0.grad(), l1.dh) 
            l0.dW = l0.grad_W(l1.dh)
            l0.db = l0.grad_b(l1.dh)
        
        self.sequence.remove(loss_obj)

## 경사하강 학습법

In [10]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = loss_obj(network(x), y)  # Forward inference
    network.calc_gradient(loss_obj)  # Back-propagation
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [11]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=5, num_neuron=32, input=10, output=2, activation=Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.9929433115270807
Epoch 1: Test loss 0.9817925794649889
Epoch 2: Test loss 0.970773064277347
Epoch 3: Test loss 0.9599009541151322
Epoch 4: Test loss 0.9491911661069669
Epoch 5: Test loss 0.9386572544386839
Epoch 6: Test loss 0.9283113459320426
Epoch 7: Test loss 0.918164102636813
Epoch 8: Test loss 0.9082247101207139
Epoch 9: Test loss 0.8985008894533657
Epoch 10: Test loss 0.8889989303528061
Epoch 11: Test loss 0.879723742602532
Epoch 12: Test loss 0.8706789226483506
Epoch 13: Test loss 0.8618668322331997
Epoch 14: Test loss 0.8532886860037426
Epoch 15: Test loss 0.8449446452002974
Epoch 16: Test loss 0.8368339147956297
Epoch 17: Test loss 0.8289548417530433
Epoch 18: Test loss 0.8213050124071496
Epoch 19: Test loss 0.8138813473119164
Epoch 20: Test loss 0.8066801922341936
Epoch 21: Test loss 0.7996974042847989
Epoch 22: Test loss 0.7929284324650332
Epoch 23: Test loss 0.7863683921590493
Epoch 24: Test loss 0.7800121333194713
Epoch 25: Test loss 0.773854302274807
